# 웹 스크래핑을 이용하여 나만의 비서 만들기

1. 네이버에서 오늘 일산의 날씨 정보를 가져옴

2. 헤드라인 뉴스 3건

3. IT 뉴스 3건

4. 해커스 어학원 홈페이지에서 오늘의 영어 회화 지문을 가져옴

In [1]:
import requests
from bs4 import BeautifulSoup
import re

### 1. 네이버에서 오늘 일산의 날씨 정보

In [34]:
def create_soup(url):
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36"}
    res = requests.get(url, headers=headers)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, "lxml")
    return soup

def print_news(index, title, link):
    print("{}. {}".format(index+1, title))
    print("  (링크) : {}".format(link))

    
def scrape_weather():
    print("[오늘의 날씨]")
    url = "https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query=%EC%9D%BC%EC%82%B0+%EB%82%A0%EC%94%A8"
    soup = create_soup(url)
    
    # 현재온도 최고/최저 온도
    cast = soup.find("p", attrs={"class":"cast_txt"}).get_text()
    curr_temp = soup.find("p", attrs={"class":"info_temperature"}).get_text().replace("도씨","")
    min_temp = soup.find("span", attrs={"class":"min"}).get_text()
    max_temp = soup.find("span", attrs={"class":"max"}).get_text()
    
    # 오전 오후 강수 확률
    morning_rain_rate = soup.find("span", attrs={"class":"point_time morning"}).get_text().strip()
    afternoon_rain_rate = soup.find("span", attrs={"class":"point_time afternoon"}).get_text().strip()
    
    # 미세먼지
    dust = soup.find("dl", attrs={"class":"indicator"})
    pm10 = dust.find_all("dd")[0].get_text()
    pm25 = dust.find_all("dd")[1].get_text()
    
    
    # 출력
    print(cast)
    print("현재 {} (최저 {} / 최고 {})".format(curr_temp, min_temp, max_temp))
    print("오전 {} / 오후 {}".format(morning_rain_rate, afternoon_rain_rate))
    print()
    print("미세먼지 {}".format(pm10))
    print("초미세먼지 {}".format(pm25))
    print()

    
def scarpe_headline_news():
    print("[헤드라인 뉴스]")
    url = "https://news.naver.com"
    soup = create_soup(url)
    
    news_list = soup.find("ul", attrs={"class":"hdline_article_list"}).find_all("li", limit=3) # 3개까지만 출력
    for index, news in enumerate(news_list):
        title = news.find("a").get_text().strip()
        link = url + news.find("a")['href']
        
        print_news(index, title, link)
    print()
    
def scrape_it_news():
    print("[IT 뉴스]")
    url = "https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid1=105&sid2=230"
    soup = create_soup(url)
    
    news_list = soup.find("ul", attrs={"class":"type06_headline"}).find_all("li", limit=3)
    for index, news in enumerate(news_list):
        a_idx = 0
        img = news.find("img")
        if img:
            a_idx = 1  # img 태그가 있으면 1번째 img 태그의 정보를 사용
        
        a_tag = news.find_all("a")[a_idx]
        title = a_tag.get_text().strip()
        link = a_tag["href"]
        
        print_news(index, title, link)
    print()

def scrape_english():
    print("[오늘의 영어 회화]")
    url = "https://www.hackers.co.kr/?c=s_eng/eng_contents/I_others_english"
    soup = create_soup(url)
    
    sentences = soup.find_all("div", attrs={"id":re.compile("^conv_kor_t")})
    print("영어 지문")
    for sentence in sentences[len(sentences)//2:]:# 8문장이 있다고 가정할 때, index 기준 4~7까지 잘라서 기져옴
        print(sentence.get_text().strip())
    print("-"*50)    
    print("한글 지문")
    for sentence in sentences[:len(sentences)//2]:
        print(sentence.get_text().strip())
        
        
if __name__ == "__main__":
    scrape_weather()
    scarpe_headline_news()
    scrape_it_news()
    scrape_english()

[오늘의 날씨]
구름많음, 어제보다 3˚ 높아요
현재 20℃  (최저 18˚ / 최고 28˚)
오전 강수확률 30% / 오후 강수확률 10%

미세먼지 12㎍/㎥좋음
초미세먼지 9㎍/㎥좋음

[헤드라인 뉴스]
1. 이상반응 1000만원 지원, 심근염·심낭염·길랭-바레 등 확대
  (링크) : https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=102&oid=018&aid=0005032933
2. 플랫폼에서 대출·펀드 손쉽게 구매…결국은 소비자 피해로 돌아온다
  (링크) : https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=101&oid=028&aid=0002559928
3. "남성 일색 내각 반대" 아프간 女시위대에 채찍…말 뿐인 '탈레반 2.0'
  (링크) : https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=104&oid=421&aid=0005593446

[IT 뉴스]
1. KT의 지니뮤직, '밀리의 서재' 품는다
  (링크) : https://news.naver.com/main/read.naver?mode=LS2D&mid=shm&sid1=105&sid2=230&oid=015&aid=0004603152
2. [르포] 수소모빌리티+쇼 개막 이틀째.. 각 기업들 "수소벨류체인 구축에 사활 건다"
  (링크) : https://news.naver.com/main/read.naver?mode=LS2D&mid=shm&sid1=105&sid2=230&oid=092&aid=0002233305
3. SK이노베이션, 에코프로 그룹과 고성능 배터리 시장 선점 위해 ’맞손’
  (링크) : https://news.naver.com/main/read.naver?mode=LS2D&mid=shm&sid1=105&sid2=230&oid=092&aid=0002233304

[오늘의 영어 회화]
영어 지문
D